In [183]:
import pandas as pd
import pulp
from pulp import GLPK
import numpy as np

In [184]:
df = pd.read_csv("Dea_data2.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 13 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Company                                         25 non-null     object 
 1   Year                                            25 non-null     int64  
 2   GHG_Intensity                                   25 non-null     float64
 3   Energy_Intensity_per_Sales                      25 non-null     float64
 4   NonRenewable_Energy_Consumption_and_Production  25 non-null     float64
 5   Water_Intensity_Per_Sales                       25 non-null     float64
 6   NonRecycled_Waste_ration                        25 non-null     float64
 7   ESG_Score                                       25 non-null     float64
 8   GHG_1                                           25 non-null     float64
 9   GHG_2                                        

In [185]:
df.replace(0, 0.01, inplace=True)
df.head()

,Company,Year,GHG_Intensity,Energy_Intensity_per_Sales,NonRenewable_Energy_Consumption_and_Production,Water_Intensity_Per_Sales,NonRecycled_Waste_ration,ESG_Score,GHG_1,GHG_2,GHG_3,1_plus_2,1_plus_2_plus_3
0,SONY US Equity,2023,283.35,96.22,89.9,0.010,3.3,5.09,2.72,8.73,250.54,11.45,261.99
1,SONY US Equity,2022,255.60,95.00,95.1,3.392,2.0,5.03,2.63,12.28,227.68,14.92,242.60
2,SONY US Equity,2021,283.36,40.75,94.4,6.784,2.0,4.97,2.36,10.44,226.74,12.80,239.54
3,SONY US Equity,2020,237.57,40.61,95.8,10.176,2.1,4.79,2.41,15.22,190.47,17.63,208.09
4,SONY US Equity,2019,263.10,40.76,98.1,13.568,4.6,4.51,2.48,15.58,216.95,18.05,235.00


In [186]:
df.shape

(25, 13)

In [187]:
class DEA_models:
    def __init__(self):
        DEFALUT_inputs_t1 = np.array(df[df['Year'] == 2019][['Energy_Intensity_per_Sales', 'NonRenewable_Energy_Consumption_and_Production',
                                'Water_Intensity_Per_Sales', 'NonRecycled_Waste_ration']])
        DEFALUT_outputs_t1 = np.array(df[df['Year'] == 2019][['GHG_Intensity', 'ESG_Score']])
        self.num_dmus = DEFALUT_inputs_t1.shape[0]
        self.num_inputs = DEFALUT_inputs_t1.shape[1]
        self.num_outputs = DEFALUT_outputs_t1.shape[1]

        self.result_matrix = pd.DataFrame(columns=['SONY', 'APPL', 'DELL', 'HPE', 'SAMSUNG', 'EorT', 'Period'])
        self.Malquist_models = []
        self.Malquist_concepted_model=[]

        # for SBM model
        self.S_minus_list = []
        #self.S_plus_list = []
        self.rho_list = []
        self.SBM_result = pd.DataFrame(columns=['Conpany', 'Year', 'Rho'])

    # the Efficiency has been moved to function calculate_efficiency_from_concept
    """
    def calculate_efficiency(self, inputs, outputs, reference_inputs, reference_outputs,i):
        model = pulp.LpProblem("Efficiency_Calculation", pulp.LpMaximize)
        lambdas = [pulp.LpVariable(f"lambda_{j}", lowBound=0) for j in range(self.num_dmus)]
        model += pulp.lpSum([lambdas[j] * reference_outputs[j, 0] for j in range(self.num_dmus)])
        for k in range(self.num_inputs):
            model += pulp.lpSum([lambdas[j] * reference_inputs[j, k] for j in range(self.num_dmus)]) <= inputs[i, k]
        model += pulp.lpSum(lambdas) == 0.1
        # model += pulp.lpSum(lambdas) == 1

        # collect models dor debug
        self.Malquist_models.append(model)
        
        model.solve()
            
        if model.status == pulp.LpStatusOptimal:
            self.efficiency_score = pulp.value(pulp.lpSum([lambdas[j] * reference_outputs[j, 0] for j in range(self.num_dmus)])) / outputs[i, 0]
        else:
            self.efficiency_score = -1
            
        return self.efficiency_score
    """

    def calculate_efficiency_from_concept(self, inputs, outputs, reference_inputs, reference_outputs, i):
        model = pulp.LpProblem("Efficiency_Calculation_from_concept", pulp.LpMinimize)

        lambdas = [pulp.LpVariable(f"lambda_{j}", lowBound=0) for j in range(self.num_dmus)]
        Theta = pulp.LpVariable("Theta", lowBound=0) 
   
        model += Theta
        for k in range(self.num_inputs):
            model += (pulp.lpSum([lambdas[j] * reference_inputs[j, k] for j in range(self.num_dmus)]) <= Theta * inputs[i, k])
        for k in range(self.num_outputs):
            model += (pulp.lpSum([lambdas[j] * reference_outputs[j, k] for j in range(self.num_dmus)]) >= outputs[i, k])

        self.Malquist_concepted_model.append(model)
        model.solve()

        if pulp.LpStatus[model.status] == "Optimal":
            efficiency_score = Theta.value()
        else:
            efficiency_score = -1
        return efficiency_score

    def return_periods(self, year_):
        inputs_t1 = np.array(df[df['Year'] == year_][['Energy_Intensity_per_Sales', 'NonRenewable_Energy_Consumption_and_Production',
                            'Water_Intensity_Per_Sales', 'NonRecycled_Waste_ration']])
        outputs_t1 = np.array(df[df['Year'] == year_][['GHG_Intensity', 'ESG_Score']])
        inputs_t2 = np.array(df[df['Year'] == (year_+1)][['Energy_Intensity_per_Sales', 'NonRenewable_Energy_Consumption_and_Production',
                            'Water_Intensity_Per_Sales', 'NonRecycled_Waste_ration']])
        outputs_t2 = np.array(df[df['Year'] == (year_+1)][['GHG_Intensity', 'ESG_Score']])
        
        return inputs_t1, outputs_t1, inputs_t2, outputs_t2

    def Malquist_result(self):
        collection_slices = []
        for year_ in [2019, 2020, 2021, 2022]:
            in_t1, out_t1, in_t2, out_t2 = self.return_periods(year_)

            malmquist_indices = []
            EC_list = []
            TC_list = []
            for Dmu_index in range(self.num_dmus):
                """
                eff_t1_t1 = self.calculate_efficiency(in_t1, out_t1, in_t1, out_t1, Dmu_index)
                eff_t1_t2 = self.calculate_efficiency(in_t1, out_t1, in_t2, out_t2, Dmu_index)
                eff_t2_t1 = self.calculate_efficiency(in_t2, out_t2, in_t1, out_t1, Dmu_index)
                eff_t2_t2 = self.calculate_efficiency(in_t2, out_t2, in_t2, out_t2, Dmu_index)
                #print(eff_t1_t1, eff_t1_t2, eff_t2_t1, eff_t2_t2)
                """
                # Test another way to calculate the efficiency
                eff_t1_t1 = self.calculate_efficiency_from_concept(in_t1, out_t1, in_t1, out_t1, Dmu_index)
                eff_t1_t2 = self.calculate_efficiency_from_concept(in_t1, out_t1, in_t2, out_t2, Dmu_index)
                eff_t2_t1 = self.calculate_efficiency_from_concept(in_t2, out_t2, in_t1, out_t1, Dmu_index)
                eff_t2_t2 = self.calculate_efficiency_from_concept(in_t2, out_t2, in_t2, out_t2, Dmu_index)
                
                if all(v is not None for v in [eff_t1_t1, eff_t2_t2, eff_t1_t2, eff_t2_t1]):
                    EC = eff_t2_t2 / eff_t1_t1 if eff_t1_t1 != 0 else None
                    TC = np.sqrt((eff_t1_t2 * eff_t2_t2) / (eff_t1_t1 * eff_t2_t1)) if all(v != 0 for v in [eff_t1_t1, eff_t2_t1]) else None
                    #print(EC, TC)
                    EC_list.append(EC)
                    TC_list.append(TC)
                    malmquist_index = EC * TC if EC is not None and TC is not None else None
                    malmquist_indices.append(malmquist_index)
                else:
                    malmquist_indices.append(None)

            malmquist_indices.extend(['ETC',f'{year_}_{year_+1}'])
            collection_slices.append(malmquist_indices)
            EC_list.extend(['EC',f'{year_}_{year_+1}'])
            TC_list.extend(['TC',f'{year_}_{year_+1}'])

            self.result_matrix.loc[len(self.result_matrix)] = malmquist_indices
            self.result_matrix.loc[len(self.result_matrix)] = EC_list
            self.result_matrix.loc[len(self.result_matrix)] = TC_list
    # input_oriented
    def SBM_model(self):
        SBM_IN = np.array(df[['Energy_Intensity_per_Sales', 'NonRenewable_Energy_Consumption_and_Production',
                            'Water_Intensity_Per_Sales', 'NonRecycled_Waste_ration']])
        SBM_OUT = np.array(df[['GHG_Intensity', 'ESG_Score']])
        n = len(SBM_IN)
        m = len(SBM_IN[0])
        s = len(SBM_OUT[0])

        # type in __init__
        # self.sbm_scores = []
        for DMU_NUM in range(n):
            prob = pulp.LpProblem("SBM_Model", pulp.LpMinimize)
            lambdas = [pulp.LpVariable(f"lambda_{k}", lowBound=0) for k in range(n)]
            s_minus = [pulp.LpVariable(f"s_minus_{i}", lowBound=0) for i in range(m)]
            #s_plus = [pulp.LpVariable(f"s_plus_{r}", lowBound=0) for r in range(s)]

            input_term = (1 / m) * sum([(s_minus[i]*(1.0/SBM_IN[DMU_NUM][i])) for i in range(m)])
            #output_term = (1 / s) * sum([(s_plus[r]*(1.0/SBM_OUT[DMU_NUM][r])) for r in range(s)])
            rho = 1-input_term
            prob += rho

            for i in range(m):
                prob += (pulp.lpSum([SBM_IN[k][i] * lambdas[k] for k in range(n)]) + s_minus[i] == SBM_IN[DMU_NUM][i])

            for r in range(s):
                prob += (pulp.lpSum([SBM_OUT[k][r] * lambdas[k] for k in range(n)]) >= SBM_OUT[DMU_NUM][r])
            
            prob.solve()

            if pulp.LpStatus[prob.status] == "Optimal":
                self.rho_list.append([DMU_NUM, pulp.value(rho)])
                for i in range(m):
                    print(f"s_minus_{i}:", pulp.value(s_minus[i]))
                    self.S_minus_list.append([DMU_NUM, pulp.value(s_minus[i])])
                """
                for r in range(s):
                    print(f"s_plus_{r}:", pulp.value(s_plus[r]))
                    self.S_plus_list.append(pulp.value(s_plus[r]))
                """
            else:
                print("No optimal solution found.")
        """
        for j, score in enumerate(self.rho_list):
            print(f"DMU {j+1} SBM Efficiency Score: {score}")
        """
        for i in self.rho_list:
            Company_name = df.iloc[i[0]]['Company']
            Year_ = df.iloc[i[0]]['Year']
            self.SBM_result.loc[len(self.SBM_result)] = [Company_name, Year_, i[1]]

In [188]:
Malquits_model = DEA_models()
Malquits_model.Malquist_result()


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/yanhanjun/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/31a7b20edb5a4395a2afa000c052b179-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/31a7b20edb5a4395a2afa000c052b179-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 47 RHS
At line 54 BOUNDS
At line 55 ENDATA
Problem MODEL has 6 rows, 6 columns and 34 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 6 (0) rows, 6 (0) columns and 34 (0) elements
0  Obj 0 Primal inf 2.135062 (2)
4  Obj 1
Optimal - objective value 1
Optimal objective 1 - 4 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.01

Welcome to t

In [189]:
Malquits_model.Malquist_concepted_model

[Efficiency_Calculation_from_concept:
 MINIMIZE
 1*Theta + 0
 SUBJECT TO
 _C1: - 40.76 Theta + 40.76 lambda_0 + 12.53 lambda_1 + 16.31 lambda_2
  + 27.9 lambda_3 + 155.4 lambda_4 <= 0
 
 _C2: - 98.1 Theta + 98.1 lambda_0 + 15.9 lambda_1 + 72.5 lambda_2
  + 62.9 lambda_3 + 88.1 lambda_4 <= 0
 
 _C3: - 13.568 Theta + 13.568 lambda_0 + 21.19 lambda_1 + 31.66 lambda_2
  + 67.97 lambda_3 + 1149.34 lambda_4 <= 0
 
 _C4: - 4.6 Theta + 4.6 lambda_0 + 43.8 lambda_1 + 13.125 lambda_2
  + 7.175 lambda_3 + 5 lambda_4 <= 0
 
 _C5: 263.1 lambda_0 + 108.54 lambda_1 + 193.29 lambda_2 + 327.83 lambda_3
  + 172.16 lambda_4 >= 263.1
 
 _C6: 4.51 lambda_0 + 5.14 lambda_1 + 3 lambda_2 + 4.01 lambda_3
  + 4.78 lambda_4 >= 4.51
 
 VARIABLES
 Theta Continuous
 lambda_0 Continuous
 lambda_1 Continuous
 lambda_2 Continuous
 lambda_3 Continuous
 lambda_4 Continuous,
 Efficiency_Calculation_from_concept:
 MINIMIZE
 1*Theta + 0
 SUBJECT TO
 _C1: - 40.76 Theta + 40.61 lambda_0 + 12.3 lambda_1 + 14.62 lambda_2
  + 2

In [190]:
#Malquits_model.result_matrix.to_csv("Malquist_Index.csv")
Malquits_model.result_matrix.to_csv('Malquits_model.csv')

In [191]:
Malquits_model.SBM_model()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/yanhanjun/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/f9af106698c3451fb075fc347bfaee65-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/f9af106698c3451fb075fc347bfaee65-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 170 RHS
At line 177 BOUNDS
At line 178 ENDATA
Problem MODEL has 6 rows, 29 columns and 154 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 6 (0) rows, 29 (0) columns and 154 (0) elements
Perturbing problem by 0.001% of 3047.8908 - largest nonzero change 3.0212823e-05 ( 0.00057190936%) - largest zero change 2.8195717e-05
0  Obj 0 Primal inf 5.7942556 (6) Dual inf 3067.5753 (4)
0  Obj 0 Primal inf 5.7942556 (6) Dual inf 1.5519834e+1

In [192]:
Malquits_model.SBM_result.to_csv('SBM_result.csv')